# ***IMPORTING LIBRARIES AND MODULES***

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn import set_config

In [2]:
pd.set_option('display.max_columns', None)


# ***GET DATAS***

In [3]:
url = "https://drive.google.com/file/d/1NFHZhCOxgW1bu5q32OqVIVBDtSo2Alkh/view?usp=sharing"
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = df = pd.read_csv(path)

# ***SPLIT DATAS***

In [4]:
# X and y creation
X = data
y = X.pop("Expensive")
# data splitting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# ***PREPROCESSOR***

In [5]:
# 1. defining categorical & ordinal columns
X_cat = X.select_dtypes(exclude="number").copy()
X_num = X.select_dtypes(include="number").copy()

# 2. numerical pipeline
numeric_pipe = make_pipeline(SimpleImputer())

# 3. categorical pipeline

    # # 3.1 defining ordinal & onehot columns
ordinal_cols = X_cat.columns.get_indexer(['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'KitchenQual', 'FireplaceQu', 'LotShape',
       'BsmtFinType2', 'HeatingQC', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PoolQC', 'Fence'])
ohe_cols = X_cat.columns.get_indexer(list(set(X_cat) - set(ordinal_cols)))

X_cat_ordinal = X_cat.columns[ordinal_cols]
X_cat_ohe = X_cat.columns[ohe_cols]

    ## 3.2 explicitly determine categories for ordinal encoding including "N_A"
ExterQual_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
ExterCond_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
BsmtQual_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
BsmtCond_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
BsmtExposure_cats = ["N_A", "No", "Mn", "Av", "Gd"]
BsmtFinType1_cats = ["N_A", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"]
KitchenQual_cats = ["Po", "Fa", "TA", "Gd", "Ex"]
FireplaceQu_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
LotShape_cats = ["N_A",'Reg', 'IR1', 'IR2', 'IR3']
BsmtFinType2_cats = ['N_A','Unf','LwQ','Rec','BLQ','ALQ','GLQ']
HeatingQC_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
GarageFinish_cats = ['N_A','Unf','RFn','Fin']
GarageQual_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
GarageCond_cats = ["N_A", "Po", "Fa", "TA", "Gd", "Ex"]
PoolQC_cats = ["N_A","Po", "Fa", "TA", "Gd", "Ex"]
Fence_cats = ["N_A",'NA','MnWw','GdWo','MnPrv','GdPrv']

cats_ord = [ExterQual_cats, ExterCond_cats, BsmtQual_cats, BsmtCond_cats, 
            BsmtExposure_cats, BsmtFinType1_cats, KitchenQual_cats, FireplaceQu_cats, 
            LotShape_cats,BsmtFinType2_cats,HeatingQC_cats,GarageFinish_cats,GarageQual_cats,
            GarageCond_cats,PoolQC_cats,Fence_cats]

### 3.2.2. defining the categorical encoder: a ColumnTransformer with 2 branches: ordinal & onehot
categorical_encoder = ColumnTransformer(
    transformers=[
        ("cat_ordinal", OrdinalEncoder(categories=cats_ord), ordinal_cols),
        ("cat_onehot", OneHotEncoder(handle_unknown="ignore"), ohe_cols),
    ]
)

## 3.3. categorical pipeline = "N_A" imputer + categorical encoder
categorical_pipe = make_pipeline(SimpleImputer(strategy="constant", fill_value="N_A"),
                                 categorical_encoder
                                )

# 4. full preprocessing: a ColumnTransformer with 2 branches: numeric & categorical
full_preprocessing = ColumnTransformer(
    transformers=[
        ("num_pipe", numeric_pipe, X_num.columns),
        ("cat_pipe", categorical_pipe, X_cat.columns),
    ]
)


# ***MODELLING***

## *DecisionTreeClassifier* 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier


scaler = StandardScaler()

full_pipeline = make_pipeline(full_preprocessing, 
                              #scaler,
                              DecisionTreeClassifier())

param_grid = {
    #"columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "decisiontreeclassifier__max_depth": range(2, 14, 2),
    "decisiontreeclassifier__min_samples_leaf": range(3, 12, 2)
}

search = GridSearchCV(full_pipeline,
                      param_grid,
                      cv=5,
                      scoring='accuracy',
                      verbose=1)

search.fit(X_train, y_train)

# create a dictionary to keep track of the scores of different models 
scores = {"dtree" : search.best_score_}


Fitting 5 folds for each of 30 candidates, totalling 150 fits


{'dtree': 0.9272403800300795}

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_full_pipeline = make_pipeline(full_preprocessing,
                                  StandardScaler(),
                                  KNeighborsClassifier()
                                 )

param_grid = {
    "columntransformer__num_pipe__simpleimputer__strategy":["mean", "median"],
    "kneighborsclassifier__n_neighbors": range(2, 50),
    "kneighborsclassifier__weights": ["uniform", "distance"]
}

knn_search = GridSearchCV(knn_full_pipeline,
                      param_grid,
                      cv=5,
                      scoring='accuracy',
                      verbose=1)

knn_search.fit(X_train, y_train)

scores["knn"] = knn_search.best_score_

Fitting 5 folds for each of 192 candidates, totalling 960 fits
